In [1]:
import csv
import pandas as pd
import re

# ---------- Extract ----------
df = pd.read_json("input/salaries.json")

# ---------- Transform ----------
# standardize column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# clean salary values
salary_cols = [c for c in df.columns if c.startswith("salary_grade")]
salary_cols

['salary_grade_1',
 'salary_grade_2',
 'salary_grade_3',
 'salary_grade_4',
 'salary_grade_5',
 'salary_grade_6',
 'salary_grade_7',
 'salary_grade_8',
 'salary_grade_9',
 'salary_grade_10',
 'salary_grade_11',
 'salary_grade_12',
 'salary_grade_13',
 'salary_grade_14']

In [2]:
def clean_salary(x):
    if pd.isna(x) or str(x).strip() == "":
        return None
    return float(re.sub(r"[^\d.]", "", str(x)))

df[salary_cols] = df[salary_cols].map(clean_salary)

# preserve original input order for sorting
df["_order"] = range(len(df))

# melt wide to long format (salary_grade_1, salary_grade_2, ... -> grade, amount)
id_vars = ["jurisdiction", "job_code", "_order"]
df = df.melt(
    id_vars=id_vars,
    value_vars=salary_cols,
    var_name="grade",
    value_name="amount",
)

# extract grade number (salary_grade_1 -> 1)
df["grade"] = df["grade"].str.replace("salary_grade_", "", regex=False).astype(int)

# drop rows with no amount
df = df.dropna(subset=["amount"])

# normalize job_code: strip leading zeros
df["job_code"] = df["job_code"].astype(str).str.lstrip("0").astype(int)

# jurisdiction mapping (kerncounty -> sdcounty per expected output)
df["jurisdiction"] = df["jurisdiction"].replace("kerncounty", "sdcounty")

# sort by original input order, then grade
df = df.sort_values(["_order", "grade"]).drop(columns=["_order"]).reset_index(drop=True)

# add id column
df.insert(0, "id", range(1, len(df) + 1))

# select output columns
df = df[["id", "jurisdiction", "job_code", "grade", "amount"]]
df

,id,jurisdiction,job_code,grade,amount
0,1,sanbernardino,1297,1,70.38
1,2,sanbernardino,1297,2,101.0
2,3,ventura,9111,1,3119.39
3,4,ventura,9111,2,4375.47
4,5,sanbernardino,10019,1,73.87
5,6,sanbernardino,10019,2,106.01
6,7,sanbernardino,1410,1,96.77
7,8,sanbernardino,1410,2,138.0
8,9,ventura,2181,1,6164.05
9,10,ventura,2181,2,8630.52


In [3]:
# ---------- Load ----------
# Match expected format: quoted header, unquoted data, amount with 2 decimals
with open("output/salaries_clean.csv", "w", newline="") as f:
    header_writer = csv.writer(f, quoting=csv.QUOTE_ALL, lineterminator="\n")
    header_writer.writerow(["id", "jurisdiction", "job_code", "grade", "amount"])
    data_writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL, lineterminator="\n")
    for _, row in df.iterrows():
        data_writer.writerow(
            [row["id"], row["jurisdiction"], row["job_code"], row["grade"], f'{row["amount"]:.2f}']
        )
print("✅ salaries_clean.csv created")

✅ salaries_clean.csv created
